# Setting environment and preparing url
- Library Installation (pip)
- Dependency Imports
- Resource Downloads



In [1]:
!pip install requests

In [2]:
# it crash sometime so i do this command.
#!rm -rf dataset/

In [3]:
!pip install requests
import os
import requests
import re
import numpy as np
import pandas as pd
import nltk
import collections
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from collections import Counter
import matplotlib.pyplot as plt
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
nltk.download('stopwords')
nltk.download('punkt_tab')



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

# Difined dataset
- Compares two different themes in novels—slavery versus romantic love stories—to see whether any interesting patterns emerge.


In [4]:
#Set books to two fictions
books_dataset = {
    "Slave_Fiction": [
        ("TheGaries", "https://www.gutenberg.org/cache/epub/11214/pg11214.txt"),
        ("UncleTom", "https://www.gutenberg.org/cache/epub/203/pg203.txt"),
        ("Clotelle", "https://www.gutenberg.org/cache/epub/241/pg241.txt")
    ],
    "Romance_Fiction": [
        ("Emma", "https://www.gutenberg.org/cache/epub/158/pg158.txt"),
        ("Innocence", "https://www.gutenberg.org/cache/epub/541/pg541.txt"),
        ("Wuthering", "https://www.gutenberg.org/cache/epub/768/pg768.txt")
    ]
}


#Remove header/footer, clean the noise
 Creat function (tool) for select only body of the books.

In [5]:
# Remove Header,Footer
# Remove Illustration tags using re.sub (Find and replace(replace with nothing("")))
# The books may be not contrain Illustration photo but just to be safe
def clean_text(text):

    if "*** START" in text:
        text = re.split(r'\*\*\* START OF.*? \*\*\*', text)[-1]
    if "*** END" in text:
        text = re.split(r'\*\*\* END OF.*? \*\*\*', text)[0]

    book_content = re.sub(r'\[Illustration.*?\]', '', text)
    return book_content.strip()



In [6]:
# use os.makedirs to create dataset folder (get the categories pass (slave and romance.)
# Second for loop, get url, use utf-8(for safety in case some non english writing style)
# split by the chapters to break the docs, it is most manageable way even the length of each chapter may different
# However some books, did not use 'chapters'for each section, it use I,II IV, the romance numbers, so I write if-else for both options
for categories, books_lists in books_dataset.items():
    print(f"{categories}")
    os.makedirs(f"dataset/{categories}", exist_ok=True)
    doc_count = 1

    for name, url in books_lists:
        print(f"  {name}")

        try:
            r = requests.get(url)
            r.encoding = 'utf-8'  # handles incoming data from web to Python
            raw_from_web = r.text  # use r.text to look in the link and translate the data into a readable string of characters (text)

            cleaned_content = clean_text(raw_from_web)  # calling function clean_text that we prepared above, so we get only body(content) of the books.

            if "CHAPTER" in cleaned_content.upper():
                chapters = re.split(r'(?i)^\s*CHAPTER\s+[IVXLCDM\d]+', cleaned_content, flags=re.MULTILINE)
            else:
                chapters = re.split(r'(?i)^\s*[IVXLCDM]+\.?\s*$', cleaned_content, flags=re.MULTILINE)

            # print(f"{name}: there is {len(chapters)} chapters")
            # there are too many docs much more than all original chapter from all books combine.
            # I write code to only keep real chapter/doc, using len to filter "chapter" word  from TOC or preface out
            # go to folder abd creat text file and lebel it, like Emma_1
            for chap in chapters:
                words = chap.split()

                if len(words) > 500:
                    filename = f"dataset/{categories}/{name}_{doc_count}.txt"
                    #open file, write(creat new file)
                    #f.write is equal open and paste the chapter we loop into file, clean it with. strip
                    with open(filename, "w", encoding="utf-8") as f:
                        f.write(chap.strip())
                    doc_count += 1
        except Exception as e:
            print(f" Error with {name}: {e}")


Slave_Fiction
  TheGaries
  UncleTom
  Clotelle
Romance_Fiction
  Emma
  Innocence
  Wuthering


#How many docs we have from each categories

In [7]:
!ls dataset/Slave_Fiction | wc -l
!ls dataset/Romance_Fiction | wc -l

120
123


#Verify the quality of docs

In [8]:
categories = ['Slave_Fiction', 'Romance_Fiction']

for group in categories:
    folder_path = f'dataset/{group}'
    files = sorted(os.listdir(folder_path))

    print(f"\n{len(files)} files in {group}")
    small_chapters = 0
    total_words = 0

    for filename in files:
        with open(os.path.join(folder_path, filename), "r", encoding="utf-8") as f:   # We are bringing data FROM the file INTO Python+ # encoding="utf-8" tells Python how to translate the file's bytes INTO text.
            count = len(f.read().split())
            total_words += count
            if count < 500:
                print(f"{filename} is too short ({count} words)")
                small_chapters += 1

    avg = total_words / len(files) if files else 0
    print(f"Average words per file: {int(avg)}")
    print(f"Number of file <500 words: {small_chapters}")


120 files in Slave_Fiction
Average words per file: 2953
Number of file <500 words: 0

123 files in Romance_Fiction
Average words per file: 3045
Number of file <500 words: 0


# We have docs ready for NLP steps

# Text Normalization

In [38]:
#lowered case, removed junk(numbers too), and stemmed words
# preparing tools
stemmer = PorterStemmer()
stop_words = set(stopwords.words('english'))

#lebel our books genre
categories = {
    'Slave_Fiction': 0,
    'Romance_Fiction': 1
}

alldata = []   # to hold the final clean text
fiction_group_label = [] # to hold the labels

#this just create the tools pipeline, but we did not loop to each docs yet.
def text_normalize_pipeline(raw_text):
    text = raw_text.lower()
    text = re.sub(r'[^a-z\s]', ' ', text) # this should use '' ? or ' '?, i go with ' 'it have a word like slave-hunt, shoud it be slavehunt? or slave hunt?
    words = text.split()

    # Keep the word that has more than 2 letters, this will be my experiment too, between cut 2 lens and not cut.
    clean_words = []
    for w in words:
        if w not in stop_words and len(w) > 2:
            stemmed_word = stemmer.stem(w)
            clean_words.append(stemmed_word)
    return " ".join(clean_words)

#load and process
for category, label in categories.items():
    folder_path = f"dataset/{category}"
    files = sorted(os.listdir(folder_path))

    print(f"{category} ({len(files)} docs)")

    for filename in files:
        file_path = os.path.join(folder_path, filename)

        with open(file_path, "r", encoding="utf-8") as f:
            chapter_doc_text = f.read()
            final_doc_text = text_normalize_pipeline(chapter_doc_text) #Text (docs) Normalization is happending herr, remember we created def at earlier loop
            alldata.append(final_doc_text)
            fiction_group_label.append(label)

#finaltexts are in alldata(kind of folders).
print(f"Loop through {len(alldata)} documents")
print(f"Sample Output: '{alldata[0][:200]}...'")

Slave_Fiction (120 docs)
Romance_Fiction (123 docs)
Loop through 243 documents
Sample Output: 'slave hunt parson delight even cloudless day set sun reflect golden ray surround hill cover beauti greensward luxuri verdur form constant garb tropic steamer columbia ran dock natchez began unload car...'


#Calculating Probabilities and Log-Likelihood

In [10]:
# use CountVectorizer counts every word in every document
# creat big bag(matrix) using fit_transform
# but we needs 2 bags, use the lebel eariler(slave=0, romance =1) for category to separate the matrix
# sum up word in each catagory
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(alldata)
vocab = vectorizer.get_feature_names_out()


X_slave = X[np.array(fiction_group_label) == 0]
X_romance = X[np.array(fiction_group_label) == 1]

slave_word_counts = np.array(X_slave.sum(axis=0)).flatten()
romance_word_counts = np.array(X_romance.sum(axis=0)).flatten()


In [11]:
# Calculate Probabilities with Laplace Smoothing (+1)
# formula 1 :  (word count + 1) / (total words in category + Vocab Size)
# P(word | category) = (count + 1) / (total_words + vocab_size)
total_slave_words = slave_word_counts.sum()
total_romance_words = romance_word_counts.sum()
vocab_size = len(vocab)

prob_word_given_slave = (slave_word_counts + 1) / (total_slave_words + vocab_size)
prob_word_given_romance = (romance_word_counts + 1) / (total_romance_words + vocab_size)


# llr= Log(P(w|Slave) / P(w|Romance))
# Positive Score = Slave Fiction
# Negative Score = Romance Fiction
llr = np.log(prob_word_given_slave / prob_word_given_romance)


In [12]:
# create df to view results of llr
df = pd.DataFrame({
    'word': vocab,
    'freq_slave': slave_word_counts,
    'freq_romance': romance_word_counts,
    'llr': llr
})

In [13]:

# change.head to 50 or 100 if you like to see more

print(df.sort_values('llr', ascending=False).head(20)[['word', 'llr', 'freq_slave', 'freq_romance']].to_string(index=False))
print(df.sort_values('llr', ascending=True).head(20)[['word', 'llr', 'freq_slave', 'freq_romance']].to_string(index=False))

   word      llr  freq_slave  freq_romance
  clare 5.953434         374             0
    eva 5.772711         312             0
ophelia 5.753356         306             0
 steven 5.726952         298             0
     ma 5.678997         284             0
    tom 5.594853         785             2
   gari 5.502972         238             0
   elli 5.451458         226             0
  legre 5.392484         213             0
 nigger 5.339714         202             0
  haley 5.278782         190             0
    sam 5.246864         184             0
 shelbi 5.202658         176             0
  cassi 5.168172         170             0
  eliza 5.138496         165             0
  kinch 5.114104         161             0
clarenc 5.095412         158             0
  negro 5.089103         157             0
 clotel 5.082754         156             0
    dat 5.063461         153             0
      word       llr  freq_slave  freq_romance
      emma -6.736221           0           864
   

In [27]:
# create a list of the names I want to hide
words_to_ignore0 = [
    'clare', 'ophelia', 'eva', 'steven', 'tom', 'gari', 'elli', 'legre',
    'haley', 'sam', 'shelbi', 'cassi', 'eliza', 'clotel', 'clarenc', 'kinch', 'caddi',
    'emma', 'archer', 'heathcliff', 'weston', 'catherin', 'linton', 'elton',
    'knightley', 'olenska', 'churchil', 'fairfax', 'beaufort', 'hareton',
    'newland', 'mingott', 'hartfield', 'luyden', 'earnshaw', 'cathi', 'highburi'
]

# filter the DataFrame by show me rows where the word is NOT in the ignore list"
df_clean = df[~df['word'].isin(words_to_ignore0)]

print("Top words: Slave fiction (Some mames removed)")
print(df_clean.sort_values('llr', ascending=False).head(10)[['word', 'llr', 'freq_slave', 'freq_romance']].to_string(index=False))
print("Top words: Romance fiction (Some names removed)")
print(df_clean.sort_values('llr', ascending=True).head(10)[['word', 'llr', 'freq_slave', 'freq_romance']].to_string(index=False))

Top words: Slave fiction (Some mames removed)
  word      llr  freq_slave  freq_romance
    ma 5.678997         284             0
nigger 5.339714         202             0
 negro 5.089103         157             0
   dat 5.063461         153             0
 chloe 4.996321         143             0
 topsi 4.939163         135             0
 jerom 4.894043         129             0
trader 4.762707         113             0
walter 4.753896         225             1
 emili 4.680469         104             0
Top words: Romance fiction (Some names removed)
    word       llr  freq_slave  freq_romance
woodhous -5.738683           0           318
 welland -5.091486           0           166
 harriet -4.815708           3           506
   edgar -4.744176           0           117
  martin -4.568612           0            98
  randal -4.495280           0            91
   nelli -4.473302           0            89
countess -4.450829           0            87
   grang -4.450829           0         

# LDA_Baseline

In [39]:
from sklearn.decomposition import LatentDirichletAllocation




# n_components = 4 = find me 4 distinct topics.
# random_state = 42 ensures you get the same results every time you run it
lda = LatentDirichletAllocation(n_components=4, random_state=42) # can change number 4 here to 10 if would like to see more

# fit the model to  Bag-of-Words Matrix (our metrix X)
lda.fit(X)

# for printing
def print_topic_words(model, feature_names, n_top_words):
    for topic_index, topic in enumerate(model.components_):
        message = f"Topic{topic_index}: "
        message += " ".join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)
    print()


print("THE TOPICS")

print_topic_words(lda, vectorizer.get_feature_names_out(), 10) # can change number 10 here for longer words

THE TOPICS
Topic0: said one tom look know well come would man hand
Topic1: clotel jerom isabella slave henri old daughter woman negro miller
Topic2: would said one heathcliff linton look come catherin could answer
Topic3: mr would could one said emma must miss look archer



#LDA_Experiment

In [33]:
from sklearn.feature_extraction import text # <--- IMPORT THIS

# I think we dont need name
word_ignore1 = ['clare', 'ophelia', 'eva', 'steven', 'tom', 'gari', 'elli', 'legre',
    'haley', 'sam', 'shelbi', 'cassi', 'eliza', 'clotel', 'clarenc', 'kinch',
    'caddi', 'chloe', 'topsy', 'walter', 'georg', 'henri', 'miller',
    'emma', 'archer', 'heathcliff', 'weston', 'catherin', 'linton', 'elton',
    'knightley', 'olenska', 'churchil', 'fairfax', 'beaufort', 'hareton',
    'newland', 'mingott', 'hartfield', 'luyden', 'earnshaw', 'cathi', 'highburi',
    'edgar', 'harriet', 'martin', 'hindley', 'isabella',
    'woodhous', 'jane', 'charli', 'welland', 'franke'
]


# I selected more words that cluttering in baselineoutput+some vague words.
# It could have more that you can cut like 'good', 'speak', 'talk',told','said, or may be some word i shoud not take it out like 'see', 'made' or  take it out for experiment
word_ignore2 = [
    'said', 'one', 'would', 'could', 'mr', 'miss', 'mrs', 'man', 'know',
    'like', 'well', 'come', 'see',  'made', 'littl', 'think', 'say','shall'
    'much', 'must', 'look', 'back', 'time', 'let','go', 'come','do'
]
#combine word_ignore1 and word_ignore2
word_ignore3 = ['clare', 'ophelia', 'eva', 'steven', 'tom', 'gari', 'elli', 'legre',
    'haley', 'sam', 'shelbi', 'cassi', 'eliza', 'clotel', 'clarenc', 'kinch',
    'caddi', 'chloe', 'topsy', 'walter', 'georg', 'henri', 'miller',
    'emma', 'archer', 'heathcliff', 'weston', 'catherin', 'linton', 'elton',
    'knightley', 'olenska', 'churchil', 'fairfax', 'beaufort', 'hareton',
    'newland', 'mingott', 'hartfield', 'luyden', 'earnshaw', 'cathi', 'highburi',
    'edgar', 'harriet', 'martin', 'hindley', 'isabella',
    'woodhous', 'jane', 'charli', 'welland', 'franke',
    'said', 'one', 'would', 'could', 'mr', 'miss', 'mrs', 'man', 'know',
    'like', 'well', 'come', 'see',  'made', 'littl', 'think', 'say','shall'
    'much', 'must', 'look', 'back', 'time', 'let','go', 'come','do'
]


# Union means: "Take Standard English + My Names + Fiction Words"

stop_list_names_only = list(text.ENGLISH_STOP_WORDS.union(word_ignore3)) # can chang to differnt word groups here if wnat to filter only names out
# run vectorizer with our filter list above
vectorizer_exp = CountVectorizer(stop_words=stop_list_names_only)
X_exp = vectorizer_exp.fit_transform(alldata)

lda_exp = LatentDirichletAllocation(n_components=4, random_state=42)
lda_exp.fit(X_exp)



LatentDirichletAllocation(n_components=4, random_state=42)

In [35]:
# to print
def print_top_words(model, feature_names, n_top_words):
    for topic_index, topic in enumerate(model.components_):
        message = f"Topic {topic_index}: "
        message += " ".join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)

# the lda experiment could be names romoves or junk removed depend how to filter the code here(stop_list_names_only = list(text.ENGLISH_STOP_WORDS.union(word_ignore3)))
# currently removed names and junked

print("LDA EXPERIMENTAL")

print_top_words(lda_exp, vectorizer_exp.get_feature_names_out(), 10) #can change number 10 here for longer words

LDA EXPERIMENTAL
Topic 0: hous hand ask repli door old day answer make father
Topic 1: new young old madam york eye slave long hous peopl
Topic 2: hand good way eye child make day old thing woman
Topic 3: thing everi good thought quit feel noth friend wish day


#TF-IDF
(+LDA)

In [36]:
print("TF-IDF")

my_stop_words = list(text.ENGLISH_STOP_WORDS.union(word_ignore1))
tfidf_vectorizer = TfidfVectorizer(
    stop_words=my_stop_words,
    min_df= 30,
    max_df=0.7
)

# taking alldata(243 chapters from 2 categories) and run the matrix
tfidf_matrix = tfidf_vectorizer.fit_transform(alldata)
tfidf_feature_names = tfidf_vectorizer.get_feature_names_out()

# need this to get the topics not only keywords
# tries to find patterns across all the chapters.(find theme)
lda_tfidf = LatentDirichletAllocation(n_components=4, random_state=42).
lda_tfidf.fit(tfidf_matrix)







TF-IDF


LatentDirichletAllocation(n_components=4, random_state=42)

In [37]:
def print_top_words(model, feature_names, n_top_words):
    for topic_index, topic in enumerate(model.components_):
        print(f"Topic {topic_index}:")
        print(" ".join([feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]]))
    print()

print_top_words(lda_tfidf, tfidf_feature_names, 10)

Topic 0:
smith compliment bate frank friendship gratitud shew recommend scrupl eleg
Topic 1:
smith compliment bate frank friendship gratitud shew recommend scrupl eleg
Topic 2:
frank bate miss randal smith mr surpriz bodi shew quit
Topic 3:
mr miss mother father slave master child door new woman



In [20]:
# ai help me here on weight output.
def print_top_words_with_scores(model, feature_names, n_top_words):
    for topic_index, topic in enumerate(model.components_):
        print(f"\nTopic {topic_index}:")
        # get the top indices just like before
        top_indices = topic.argsort()[:-n_top_words - 1:-1]

        # loop through and print the word and the score
        for i in top_indices:
            word = feature_names[i]
            score = topic[i]
            print(f"  {word}: {score:.2f}")


print_top_words_with_scores(lda_tfidf, tfidf_feature_names, 10)


Topic 0:
  smith: 0.25
  compliment: 0.25
  bate: 0.25
  frank: 0.25
  friendship: 0.25
  gratitud: 0.25
  shew: 0.25
  recommend: 0.25
  scrupl: 0.25
  eleg: 0.25

Topic 1:
  smith: 0.25
  compliment: 0.25
  bate: 0.25
  frank: 0.25
  friendship: 0.25
  gratitud: 0.25
  shew: 0.25
  recommend: 0.25
  scrupl: 0.25
  eleg: 0.25

Topic 2:
  frank: 6.06
  bate: 5.21
  miss: 3.96
  randal: 3.38
  smith: 3.22
  mr: 2.99
  surpriz: 2.63
  bodi: 2.37
  shew: 2.10
  quit: 1.84

Topic 3:
  mr: 25.03
  miss: 15.43
  mother: 11.79
  father: 11.65
  slave: 10.67
  master: 10.28
  child: 10.09
  door: 9.90
  new: 9.47
  woman: 9.36
